# Welcome to ProgPy's Sim Pump Example

In this example notebook, we'll create a [Centrifugal Pump](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#pump-model) model and use our [Simulation](https://nasa.github.io/progpy/prog_models_guide.html#simulation) methods to simulate until threshold is met.

This model simulates the behavior of a centrifugal pump as it operates and wears over time, leading to potential failures. There are four types of events that the model associates with failure:

- `ImpellerWearFailure`: This event represents failure of the impeller due to wear.
- `PumpOilOverheat`: This event occurs when the pump oil overheats.
- `RadialBearingOverheat`: This event occurs when the radial bearing overheats.
- `ThrustBearingOverheat`: This event occurs when the thrust bearing overheats.

### Importing Modules

In [ ]:
from progpy.models import CentrifugalPump
import matplotlib.pyplot as plt
from progpy.sim_result import SimResult

First, we'll need to define the Centrifugal Pump object.

In [ ]:
pump = CentrifugalPump(process_noise= 0)
pump.parameters['x0']['wA'] = 0.01  # Setting Wear Rate

In the `CentrifugalPumpBase` class of ProgPy, wear rates play a significant role in modeling the degradation and eventual failure of different components of the pump. These rates represent how quickly different parts of the pump wear down or degrade over time due to operation. Here's a closer look at each wear rate:

- `wA`: This wear rate corresponds to the impeller area (`A`). It represents how quickly the impeller area degrades or wears down over time. <span style="color:teal">The impeller is a crucial part of the pump as it imparts energy to the fluid, and wear on the impeller can affect the pump's efficiency and flow rate.<span>

- `wRadial`: This wear rate is associated with the radial friction coefficient (`rRadial`). It represents how quickly friction in the radial bearings increases over time. <span style="color:teal">The radial bearings support the weight of the shaft and impeller, and increased friction can lead to increased heat and wear on these components, potentially leading to bearing failure.<span>

- `wThrust`: This wear rate pertains to the rolling friction coefficient (`rThrust`). It signifies how quickly friction in the thrust bearings increases over time. <span style="color:teal">The thrust bearings help balance axial forces in the pump, and increased friction can lead to overheating and wear, potentially resulting in bearing failure.<span>

These wear rates are incorporated into the model's state update equations, allowing the model to simulate not just the pump's behavior under normal operating conditions, but also how its performance will degrade over time due to wear. This makes the model a powerful tool for predicting when the pump might fail and planning maintenance to prevent such failures.

Next, let's define the [Future Load](https://nasa.github.io/progpy/prog_models_guide.html#future-loading) Function that we will use to simulate the pump.

In [ ]:
cycle_time = 3600
def future_loading(t, x=None):
    t = t % cycle_time
    if t < cycle_time/2.0:
        V = 471.2389
    elif t < cycle_time/2 + 100:
        V = 471.2389 + (t-cycle_time/2)
    elif t < cycle_time - 100:
        V = 571.2389
    else:
        V = 471.2398 - (t-cycle_time)

    return pump.InputContainer({
        'Tamb': 290,
        'V': V,
        'pdisch': 928654, 
        'psuc': 239179, 
        'wsync': V * 0.8
    })

The Future Load Function is a function that takes in the current state of the system and returns the future load on the system. In this case, the future load is the pump's flow rate (`Q`) and the pump's head (`H`). The pump's flow rate and head are determined by the pump's speed (`N`) and the pump's impeller diameter (`D`).

Here, the Voltage is alternating depending on the cycle timer.

In [ ]:
import numpy as np

# Define the time array
t_values = np.arange(0, 2*cycle_time, 1) # Adjust the step size as needed

# Calculate the corresponding voltage values
V_values = [future_loading(t)['V'] for t in t_values]

# Create the plot
plt.figure(figsize=(10, 5))
plt.plot(t_values, V_values)
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Voltage vs Time')
plt.grid(True)
plt.show()

In the `CentrifugalPumpBase` class of ProgPy, we have several inputs that represent the operating conditions of the pump. These inputs are crucial for simulating the pump's behavior accurately under various conditions. Here's a breakdown of each input:

- `Tamb`: This is the ambient temperature, measured in Kelvin (K). It indicates the temperature of the surrounding environment where the pump is operating. 

- `V`: This stands for the voltage, measured in volts (V). This is the electrical potential difference that is applied to the pump's motor. Changes in voltage can affect the rotational speed and power of the pump.

- `pdisch`: This is the discharge pressure, measured in Pascals (Pa). It represents the pressure of the fluid as it exits or is discharged from the pump. 

- `psuc`: This is the suction pressure, also measured in Pascals (Pa). It signifies the pressure of the fluid as it enters or is drawn into the pump.

- `wsync`: This is the synchronous rotational speed of the supply voltage, measured in radians per second (rad/sec). It reflects the frequency of the electrical power supply which can influence the rotational speed of the pump's motor.

In addition to these inputs, the model also accepts several keyword arguments representing various parameters of the pump. These parameters include atmospheric pressure (`pAtm`), empirical coefficients for the flow torque equation (`a0`, `a1`, `a2`), impeller blade area (`A`), and wear rates (`wA`, `wRadial`, `wThrust`)! These parameters are used within the model's equations to closely simulate the physical behavior of the pump.

With our future loading function defined, we can now use ProgPy's [Simulation](https://nasa.github.io/progpy/prog_models_guide.html#simulation) methods to simulate the pump until threshold is met.

We need to define the first measured output. This is needed by the `simulate_to_threshold` method to initialize state!

In [ ]:
first_output = pump.output(pump.initialize(future_loading(0),{}))
config = {
    'horizon': 1e5,
    'save_freq': 1e3,
    'print': True
}
simulated_results = pump.simulate_to_threshold(future_loading, first_output, **config)


Furthermore, we can plot the results of our simulation!

In [ ]:
simulated_results.inputs.plot(compact = False, title = 'Inputs', xlabel = 'time', ylabel = {lbl: lbl for lbl in pump.inputs}, tight_layout=True)
simulated_results.outputs.plot(compact = False, title = 'Outputs', xlabel = 'time', ylabel = '', tight_layout=True)
simulated_results.states.plot(compact = False, title = 'States', xlabel = 'time', ylabel = '', tight_layout=True)
simulated_results.event_states.plot(compact = False, title = 'Events', xlabel = 'time', ylabel = '', tight_layout=True)

thresholds_met = [pump.threshold_met(x) for x in simulated_results.states]
thresholds_met = SimResult(simulated_results.times, thresholds_met)
thresholds_met.plot(compact = False, title = 'Threshold Met', xlabel = 'time', ylabel = '', tight_layout=True)